In [187]:
import numpy as np
import random as rd
from sys import getsizeof
import math 

### State

In [188]:
def visualize_board(state):
    n = len(state)
    columns = "abcdefgh"  
    
    print("   " + "  ".join(columns[:n]))

    for row in range(n):
        line = f"{row + 1}  " 
        for col in range(n):
            if state[col] == row + 1: 
                line += "Q  "
            else:
                line += ".  "
        line += f"{row + 1}"  
        print(line)
    
    print("   " + "  ".join(columns[:n]))

# Example usage:
state = np.array([1,2,3,4,5,6,7,8])
visualize_board(state)

   a  b  c  d  e  f  g  h
1  Q  .  .  .  .  .  .  .  1
2  .  Q  .  .  .  .  .  .  2
3  .  .  Q  .  .  .  .  .  3
4  .  .  .  Q  .  .  .  .  4
5  .  .  .  .  Q  .  .  .  5
6  .  .  .  .  .  Q  .  .  6
7  .  .  .  .  .  .  Q  .  7
8  .  .  .  .  .  .  .  Q  8
   a  b  c  d  e  f  g  h


### Population Generation

In [189]:
def init_population_generation(n):
    # low while is included while high value is excluded in the calculations
    return np.random.randint(low=1, high=8+1, size=(n,8))

In [190]:
population_size = 100 # choose an even number
state = init_population_generation(population_size)
type(state)

numpy.ndarray

In [223]:
x = [1,2,3,4]
x[0], x[1] = 7, 8
print(x)

[7, 8, 3, 4]


### Fitness Function

In [191]:
def fitness_function(state):
    # To Check: 
    # 1] Queens can't be in the same row (due to state representation, no need to check for conflicting columns)
    # 2] Queens can't be in the same diagonal
    # Fitness Functions returns: '#(Conflicting Pairs of Queens)'
    n = len(state)
    conflict_counter = 0
    for Q1 in range(n):
        for Q2 in range(Q1+1, n):
            # counts conflicts row-wise
            if(state[Q1] == state[Q2]):
                conflict_counter += 1
            # counts conflicts diag-wise
            # EX: [_,1,_,_,_,_,6,_] -> (2,1) & (7,6) => DIAG: (2-7)=(1-6) <=> (-5)=(-5)
            if(state[Q1]-state[Q2] == Q1-Q2):
                conflict_counter += 1
    return conflict_counter

### Evaluate Function Fitness

In [192]:
def eval_fitness(state):
    fitness_evals = []
    if(state.ndim == 1):
            fitness_evals.append(1 - fitness_function(state) / 28) 
    elif(state.ndim == 2):
        for i in range(len(state)):
            fitness_evals.append(1 - fitness_function(state[i]) / 28) 
    return np.array(fitness_evals)

fitness_evals = eval_fitness(state)

In [193]:
state.shape

(100, 8)

### Selection

In [194]:
eval_fitness(state[38])

array([0.78571429])

In [195]:
# Tournament Strategy: "Best 2 out of Random 5"
def selection(state, population_size):
    selected_individuals = []
    while(len(selected_individuals) < population_size):
        i = np.random.randint(low=0, high=len(state))
        idxs = np.random.choice(population_size, 5, replace=False)
        nr1_individual = nr2_individual = 0
        for i in range(idxs.size):
            if(i == 0):
                if(eval_fitness(state[idxs[0]]) > eval_fitness(state[idxs[1]])):
                    nr1_individual = state[idxs[0]].copy()
                    nr2_individual = state[idxs[1]].copy()
                    continue
                else:
                    nr1_individual = state[idxs[1]].copy()
                    nr2_individual = state[idxs[0]].copy()
                    continue
            if(i == 1):
                continue
            if(eval_fitness(state[idxs[i]]) > eval_fitness(nr1_individual)):
                nr2_individual = nr1_individual.copy()
                nr1_individual = state[idxs[i]].copy()
            if(eval_fitness(state[idxs[i]]) > eval_fitness(nr2_individual)):
                nr2_individual = state[idxs[i]].copy()
        selected_individuals.extend([nr1_individual, nr2_individual])
    return np.array(selected_individuals)

In [196]:
selected_individuals = selection(state, population_size)

In [197]:
idxs = np.random.choice(100, 5, replace=False)
idxs

array([ 9, 82,  3, 18, 26])

In [198]:
print(len(selected_individuals), population_size)

100 100


In [199]:
print(selected_individuals)

[[1 1 6 5 2 5 8 6]
 [8 8 4 3 5 7 4 7]
 [7 2 4 3 8 3 1 1]
 [7 2 4 3 8 3 1 1]
 [7 5 3 5 6 2 7 4]
 [7 5 3 5 6 2 7 4]
 [8 3 3 4 7 5 1 1]
 [1 6 4 8 2 4 3 3]
 [5 8 3 6 8 6 4 2]
 [1 1 6 5 2 5 8 6]
 [8 2 4 7 3 5 4 8]
 [2 3 6 2 4 5 2 6]
 [4 4 1 3 5 6 2 2]
 [3 5 5 6 2 4 7 4]
 [7 2 5 7 3 1 8 8]
 [7 2 5 7 3 1 8 8]
 [2 8 7 4 2 1 3 8]
 [5 1 8 8 1 6 2 6]
 [4 6 2 8 3 8 7 6]
 [1 5 8 2 4 5 6 2]
 [2 3 2 8 5 8 5 5]
 [8 6 5 5 6 7 3 7]
 [2 8 7 4 2 1 3 8]
 [2 8 7 4 2 1 3 8]
 [5 3 4 2 7 6 2 7]
 [4 8 7 4 1 3 6 8]
 [4 8 7 4 1 3 6 8]
 [2 8 7 4 2 1 3 8]
 [1 1 6 5 2 5 8 6]
 [8 2 5 2 7 7 8 4]
 [8 3 3 4 7 5 1 1]
 [8 3 3 4 7 5 1 1]
 [6 8 1 3 2 4 6 4]
 [6 8 1 3 2 4 6 4]
 [5 4 6 2 1 7 3 8]
 [3 8 4 2 3 8 6 2]
 [5 8 3 6 8 6 4 2]
 [4 3 5 8 7 1 4 4]
 [3 5 3 7 6 4 8 2]
 [1 6 4 8 2 4 3 3]
 [5 8 3 6 8 6 4 2]
 [5 8 3 6 8 6 4 2]
 [6 8 1 3 2 4 6 4]
 [6 7 1 8 6 4 2 2]
 [7 6 4 6 8 5 8 1]
 [7 5 3 3 8 8 5 1]
 [8 2 5 8 7 2 5 3]
 [3 3 6 5 5 8 1 7]
 [3 6 8 1 3 5 2 1]
 [3 6 8 1 3 5 2 1]
 [3 6 8 1 3 5 2 1]
 [3 8 4 2 3 8 6 2]
 [4 8 8 3 1 

### Crossover (Recombination)

In [200]:
# state = current boards after selection
# probability = probability of mutation, usually between 70-90%
def recombination(selected_individuals, population_size, crossover_rate):
    new_population = []
    copy_selected_individuals = selected_individuals.copy()

    while len(new_population) < population_size:
        crossover_point = np.random.randint(3, 6)
        dad_idx = np.random.randint(0, len(copy_selected_individuals))
        mom_idx = dad_idx
        while(dad_idx == mom_idx):
            mom_idx = np.random.randint(0, len(copy_selected_individuals))
        dad = copy_selected_individuals[dad_idx]
        mom = copy_selected_individuals[mom_idx]

        if(np.random.rand() < crossover_rate):
            child_one = child_two = [0] * 8
            for j in range(8):
                if(j < crossover_point):
                    child_one[j] = dad[j]
                    child_two[j] = mom[j]
                else:
                    child_one[j] = mom[j]
                    child_two[j] = dad[j]
            new_population.append(child_one)
            new_population.append(child_two)
        else:
            new_population.append(dad)
            new_population.append(mom)

    return np.array(new_population[:population_size])

In [201]:
print('BEFORE:', np.average(eval_fitness(state)))
crossover_rate = 0.80
print('AFTER:', np.average(eval_fitness(recombination(selected_individuals, population_size, crossover_rate))))

BEFORE: 0.8028571428571428
AFTER: 0.840357142857143


In [202]:
arr_100 = []
for _ in range(100):
    arr_100.append(np.average(eval_fitness(recombination(selected_individuals, population_size, crossover_rate))))
print(np.average(arr_100))

0.8432035714285715


In [203]:
state = recombination(selected_individuals, population_size, crossover_rate)

In [204]:
state.shape

(100, 8)

### Mutation

In [205]:
# state = current boards after recombination
# mutation_prob = probability of mutation, usually between 1-5% (give it as a decimal, i.e. 0.03)
def mutation(state, mutation_prob):
    # experiment with different mutation strategies (single-gene mutations, swap mutations, or mix of both)
    for board in state:
        if(np.random.rand() < mutation_prob):
            new_gene = np.random.randint(1,8+1)
            insert_idx = np.random.randint(0,8)
            board[insert_idx] = new_gene
    return state

In [206]:
mutation_prob = 0.03
state = mutation(state, mutation_prob)

In [207]:
if(np.any(np.array([0.5, 0.3, 1]) == 1)):
    index = np.argmax(np.array([0.5, 0.3, 1]))
    print(index)

2
